# 2.1.2 Algebraic data types

User-defined types in object-oriented languages are specified through class or trait declarations. Types can also be specified from already existing user-defined types through the inheritance mechanism.  

In functional programming, the rules for declaring new types are different: no inheritance or classes, just _products_, _sums_ and _exponentiation_ of types. Because of the correspondence with arithmetic (which goes beyond the terminology!), these types are called **algebraic data types** (ADTs). 

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 15. Case Classes and Pattern Matching

__[Scala book (online)](https://docs.scala-lang.org/overviews/scala-book/introduction.html)__.

- [Match Expressions](https://docs.scala-lang.org/overviews/scala-book/match-expressions.html)
- [Case classes](https://docs.scala-lang.org/overviews/scala-book/case-classes.html)
- [Case objects](https://docs.scala-lang.org/overviews/scala-book/case-objects.html)

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapters 19. Functional Programming as Algebra 

[__Tony Morris on ADTs__](https://about.chatroulette.com/posts/algebraic-data-types/)

## Product types

A value of product type $T_1 * T_2$ is created with a value of $T_1$ **and** a value of $T_2$. The constructor function is:
  - `create: (T1, T2) -> T1 * T2` (create is a 2-ary function). 

Given a value of a product type, we can obtain back both values with observers:
  - `fst: T1 * T2 -> T1` 
  - `snd: T1 * T2 -> T2`
  


### Scala case classes

There are several things which are desirable to work with products/records:
 - Create new product objects without having to invoke `new`
 - Equality of product objects by value, not by reference
 - Off-the-shelf hash code
 - Pattern matching (more on this later on)


Scala can make all of this for us automatically, using so-called `case classes`: 

In [ ]:
case class Rectangle(width: Int, height: Int)
case class Circle(radius: Int)
case class Triangle(width: Int)

This declaration of the `Rectangle` record is essentially equivalent to what we did manually: the `case` keyword tells the Scala compiler to generate a companion object with an `apply` constructor; override the `equals` and `hashCode` methods, among other things.

In [ ]:
Rectangle(1,1) == Rectangle(1,1)

In [ ]:
Rectangle(1,1).hashCode

### Standard products in Scala: `TupleN` classes

The standard library of Scala has already defined for us generic case classes that represent the n-ary products (up to 22). The rough definition of `Tuple2` goes like this:

In [ ]:
object Std{
    case class Tuple2[A, B](_1: A, _2: B)
}

And Scala offers syntactic sugar, both for Tuple types and values. So, instead of writing something like this: 

In [ ]:
val t3: Tuple3[Int, String, Boolean] = Tuple3(1, "uno", true)

we can write it as follows:

In [ ]:
val a: (Int, String, Boolean) = (1, "uno", true)

### Why are products called _algebraic_?

This is an example to illustrate the analogy between algebraic data types and arithmetic.

In [ ]:
// Number of values of Boolean type: 2
true: Boolean
false: Boolean 

// Number of values of (Boolean, Boolean) type: 2 * 2 
(true, true): (Boolean, Boolean)
(true, false): (Boolean, Boolean)
(false, false): (Boolean, Boolean)
(false, true): (Boolean, Boolean)

In general, types may be regarded as sets of values. Then, the cardinal of $A * B$, for types $A$ and $B$ is: $|A * B| = |A| * |B|$.

If product types are analogous to number multiplication, then, is there any type which corresponds to the number 1, i.e. the neutral element of the multiplication? It has to be a type $1$ such that $A*1 \cong A \cong 1*A$, where the sign $\cong$ represents the _isomorphism_ of types, i.e. the types do not need to be equal but there should be a 1-1 mapping (a bijection) between the values of $A*1$ and $A$. Since the type $1$ has to comply with the identity rules, we have that $|A*1| = |A|$, but then $|A| * |1| = |A|$. So, $|1| = 1$, i.e. $1$ must be the type with just one value, i.e. the equivalent to the singleton set. 


This type already exists in the Scala standard library, and it's called `Unit`, and its only value is `()`:

In [ ]:
val unit: Unit = ()

The isomorphism $Boolean * 1 \cong Boolean$ is witnessed by the following functions: 

In [ ]:
def from(p: (Boolean, Unit)): Boolean = 
    p._1

def to(b: Boolean): (Boolean, Unit) = 
    (b, ())

which satisfy:
- `from(to(b))=b`, for all `b: Boolean`, and 
- `to(from(b))=b` for all `b: (Boolean, Unit)`.

In [ ]:
from(to(true))==true
from(to(false))==false
to(from((true,())))==(true,())
to(from((false,())))==(false,())

Note that any function that returns a value of type `Unit` is completely useless from a purely functional perspective, since we already know in advance which is the (only possible) value that it returns: `()`. Therefore, if such a function makes sense is because it does something else than returning values: it must have some side effect, i.e. it has to be _impure_. This is also why we can say that `Unit` is the Scala equivalent to Java's `void`.

## Sum types       

Besides multiplying types, we can also _sum_ types. Given types $A$ and $B$, the sum type $A + B$ represents **either** a value of type $A$ **or** a value of type $B$. Therefore, we have that 

$|A + B| = |A| + |B|$

For instance (the symbol $:=$ is used to give a name to a type):

- $MaybeInt := Int + 1$. A value of this type may be an integer; if it is not, then it is the unit value (a value that we use to signal that it is not an integer). So, $|MaybeInt| = |Int| + |1| = |Int| + 1$
- $EitherIntOrString := Int + String$. A value of this type is either an integer or a string, i.e. $|EitherIntOrString| = |Int| + |String|$
- $Shape := Circle + Rectangle + Triangle$. If we have a value of type $Shape$, then we have either a $Circle$, a $Triangle$ or a $Rectangle$. So, $|Shape| = |Circle| + |Triangle| + |Rectangle|$

We create and observe values of a sum type $A + B$ with the following functions: 
- Injection functions: 
  - `injA: A -> A + B`
  - `injB: B -> A + B`
- Match function:
  - `match: (A -> C) -> (B -> C) -> A + B -> C`
  
Note that the `match` function is a higher-order function. Basically, it says: if I know how to obtain a $C$ from $A$ (using function $A \rightarrow C$), and I know how to obtain a $C$ from $B$, then I know how to obtain a $C$ from $A+B$ (since $A+B$ is either an $A$ or a $B$).

### Sum types in Scala

How do we define sum types in an object-oriented language like Scala? Basically, we need _inheritance_, but with a special twist: it has to be _sealed_.

In [ ]:
// type Shape = Rectangle + Triangle + Circle 

sealed abstract class Shape
case class Rectangle(width: Int, height: Int) extends Shape
case class Triangle(width: Int) extends Shape
case class Circle(radius: Int) extends Shape

As we already saw, the keyword `sealed` prevents the extension of the inheritance hierarchy with new subclasses. This guarantees that the sum type will remain consistent everywhere, i.e. that whenever we have an instance of `Shape` it will be either a rectangle, a circle or a triangle, and nothing else.

We create values of type `Shape` by using the constructors of its subclasses:

In [ ]:
val s1: Shape = Rectangle(1,1)
val s2: Shape = Triangle(1)
val s3: Shape = Circle(2)

And we _observe_ these values with _pattern matching_, as follows:

In [ ]:
val s: String = s1 match {
    // Rectangle => String
    case r: Rectangle => "Rectangle" : String
    // Triangle => String
    case t: Triangle  => "Triangle"  : String 
    // Circle => String
    case c: Circle    => "Circle"    : String
}

Each `case` declaration represents a function from the corresponding type to the common target result. Being `sealed`, the compiler can check whether some pattern matching expression is complete or not, and warn us in case it's not.

In [ ]:
// This is Almond specific. 
// Also, note that it's possible that this diretive does not work for some kernel versions
interp.configureCompiler(_.settings.nowarn.value = false)

In [ ]:
// A warning should be raised here
val s: String = s1 match {
    case r: Rectangle => "R"
}

### Standard sum types in Scala

The standard library of Scala provides two important sum types: [`Option`](https://www.scala-lang.org/api/current/scala/Option.html) and [`Either`](https://www.scala-lang.org/api/current/scala/util/Either.html). They can be defined as follows: 

In [ ]:
object StdSumTypes{
    sealed abstract class Option[A]
    case class Some[A](a: A) extends Option[A]
    case class None[A]() extends Option[A]

    sealed abstract class Either[A, B]
    case class Left[A, B](a: A) extends Either[A, B]
    case class Right[A, B](b: B) extends Either[A, B]
}

These types are important for error handling. We will see how they allow us to get rid of exceptions, at least in the part of our code that we wish to be purely functional. Here it's a small example:

In [ ]:
// Using exceptions

def divideWithExceptions(a: Double, b: Double): Double =
    if (b==0) throw new Exception("divide by cero")
    else a/b

In [ ]:
// divideWithExceptions(5,0)

In [ ]:
// Using option

def divideWithOption(a: Double, b: Double): Option[Double] =
    if (b==0) Option.empty[Double] // None
    else Some(a/b)

We now return a value which indicates whether there was an error or not:

In [ ]:
val maybeDouble: Option[Double] = 
    divideWithOption(5, 0)

In [ ]:
// Using Either

def divideWithEither(a: Double, b: Double): Either[String, Double] =
    if (b==0) Left("Divide by cero")
    else Right(a/b)

And now a value which, in case of error indicates the reason:

In [ ]:
val eitherDoubleOrString: Either[String, Double] = 
    divideWithEither(5,0)

### The 0 type in Scala

If the unit type was the identity element for product types, is there any identity type for sums? It has to be a type which satisfies the following conditions: 

- $0 + A \cong A$
- $A + 0 \cong A$

But $|0 + A| = |0| + |A|$, so $|0| = 0$, i.e. the type $0$ must inhabited. In other words, it is a type such that there is no value of that type. 

We don't have to define this type in Scala it ourselves, since the identity element of sums is already defined in the Scala standard library: it's the type `Nothing`. Since we can't create instances of this type, the only thing that we can do if we have to return a value of this type, or assign a variable of this type a value, is to throw an exception:

In [ ]:
lazy val impossible: Nothing = 
    throw new Exception("no value of type Nothing")

The `???` expression in Scala means essentially an exception of type `Nothing`. Also, note that `Nothing` is the botton of the Scala inheritance hierarchy, i.e. `Nothing` is a subclass of any Scala class. That's why we can use `???` in place of any value in Scala.

In [ ]:
def i: Int = ??? // throw new Exception("no value")

The isomorphism $Int + 0 \cong Int$ is witnessed by the following functions:

In [ ]:
// IntOrNothing := Int + Nothing

sealed abstract class IntOrNothing
case class AnInt(i: Int) extends IntOrNothing
case class Impossible(n: Nothing) extends IntOrNothing

def fromInt(s: IntOrNothing): Int = 
    s match {
        case AnInt(i) => i: Int
        // case _ => (throw new Exception("It can't happen")) : Nothing // Int
    }

def toIntOrNothing(i: Int): IntOrNothing = 
    AnInt(i)

### More on pattern matching

Let's implement a function that calculates the area of a shape:

In [ ]:
import scala.math._

def area(shape: Shape): Double = 
    shape match {
        case r: Rectangle => r.width * r.height
        case c: Circle => Pi * pow(c.radius, 2)
        case t: Triangle => t.width * t.width / 2.0
    }

In [ ]:
area(Circle(1))
area(Rectangle(2,3))
area(Triangle(3))

We can implement this function more conveniently, using extractors to _deconstruct_ the value and access more directly its member attributes: 

In [ ]:
def area(shape: Shape): Double = 
    shape match {
        case Rectangle(w, h) => w * h
        case Circle(r) => Pi * pow(r, 2)
        case Triangle(w) => w * w / 2.0
    }

This can also be used in `val` declarations:

In [ ]:
val r: Rectangle = Rectangle(1,2)
val r1@Rectangle(w,h) : Rectangle = Rectangle(1,2)
val Rectangle(w1,h1) = Rectangle(1,2)

We can also use _guards_ in `case` branches:

In [ ]:
def bigShape(s: Shape): String = 
    s match {
        case Rectangle(w,h) if w+h > 10 => "big rectangle"
        case Circle(r) if r > 100 => "big circle"
        case _ => "no big shape"
    }

In [ ]:
bigShape(Rectangle(2,9))
bigShape(Circle(1))
bigShape(Triangle(2))

We can pattern match on specific _values_ of variables: 

In [ ]:
def isRectangle(r: Shape, w: Int, h: Int): Boolean = 
    r match {
        case Rectangle(`w`, `h`) => true
        case _ => false
    }

In [ ]:
isRectangle(Rectangle(1,2),1,2)

We can pattern match repeatedly until several levels of nesting:

In [ ]:
def foo(s: Either[Either[Int, String], (Int, Either[String, Boolean])]): Boolean = 
    s match {
        case Left(Left(i)) => ???
        case Left(Right(s)) => ???
        case Right((i, Left(s))) => ???
        case Right((i, Right(b))) => ???
    }

This is actually more convenient than this way:

In [ ]:
def foo(s: Either[Either[Int, String], (Int, Either[String, Boolean])]): Boolean = 
    s match {
        case Left(l) => 
            l match {
                case Left(i) => ???
                case Right(s) => ???
            }
        case Right((i, e)) => 
            e match {
                case Left(s) => ???
                case Right(b) => ???
            }
    }

Note that `(Int+String)+Int*(String+Boolean)` $\cong$ `Int + String + Int*String + Int*Boolean`, i.e. four cases.


More details on pattern matching in Scala can be found [here](https://docs.scala-lang.org/tour/pattern-matching.html). Also, note that we can create custom pattern matching expressions using so-called [_extractors_](https://docs.scala-lang.org/tour/extractor-objects.html). We won't need them in the course, but they can be really helpful to simplifiy and get more understandable pattern matching expressions. 

## Exponent types

We already know how to build new types by _adding_ and _multiplying_ other types. We will see now that function types can be properly called _exponent_ types. Indeed, let's consider how many functions are there with type `Boolean => Boolean`:

In [ ]:
// false -> false, true -> false
val f1: Boolean => Boolean = 
    (x: Boolean) => false

// false -> true, true -> true
val f2: Boolean => Boolean = 
    (x: Boolean) => true

// false -> false, true -> true 
val f3: Boolean => Boolean = 
    (x: Boolean) => x

// false -> true, true -> false
val f4: Boolean => Boolean = 
    (x: Boolean) => !x

In general, $|X\Rightarrow Y|=|Y|^{|X|}$, since for any $X$ we have $|Y|$ values available. But the correspondence with arithmetic exponents goes further, since the familiar laws of exponents:

$$ 
\begin{array}{rcl}
X^0 & = & 1 \\
1^X & = & 1 \\
X^1 & = & X \\
Z^{X+Y} & = &  Z^X * Z^Y \\
Z^{X*Y} & = & Z^{Y^X} \\
(Y*Z)^X & = & Y^X*Z^X \\
\end{array}
$$



can be recasted in terms of laws for algebraic data types as follows:

$$ 
\begin{array}{rcl}
0 \Rightarrow X & \cong & 1 \\
X \Rightarrow 1 & \cong & 1 \\
1 \Rightarrow X & \cong & X \\
(X+Y) \Rightarrow Z & \cong & (X \Rightarrow Z) * (Y \Rightarrow Z) \\
(X*Y) \Rightarrow Z & \cong & X \Rightarrow (Y \Rightarrow Z) \\
X \Rightarrow Y*Z & \cong & (X \Rightarrow Y)*(X \Rightarrow Z) \\
\end{array}
$$



And these isomorphisms hold! Let's see some examples.

## $X \Rightarrow 1 \cong 1$

Essentially, this isomorphism tells us that we only have one implementation of the function type $X \Rightarrow 1$, for any type $X$. Namely:

In [ ]:
def f[X](b: X): Unit = 
    ()

which agrees with the formula: $|X \Rightarrow 1| = |1|^{|X|} = 1$.

## $0 \Rightarrow X \cong 1$

The same happens for $0 \Rightarrow X$, for any type $X$: 

In [ ]:
def f[X](n: Nothing): X = 
    n

(recall that `Nothing <: X` for any `X` in Scala). This agrees with the arithmetic formula $|0 => X|=|X|^0=1$.

## $(Y+Z) \Rightarrow X \cong (Y \Rightarrow X) * (Z \Rightarrow X)$

We show this isomorphism by implementing the following functions:

In [ ]:
def from[X,Y,Z](a: Either[Y,Z] => X): (Y => X, Z => X) = 
    ((y: Y) => a(Left(y)), 
     (z: Z) => a(Right(z)))
    

In [ ]:
def to[X, Y, Z](a: (Y => X, Z => X)): Either[Y, Z] => X = 
    (x: Either[Y, Z]) => 
        x match { 
            case Left(y) => 
                a._1(y)
            case Right(z) => 
                a._2(z)
        }

The last pattern matching can also be written more concisely using so-called [partial functions](https://www.scala-lang.org/api/current/scala/PartialFunction.html): 

In [ ]:
def to[X, Y, Z](a: (Y => X, Z => X)): Either[Y, Z] => X = 
    { 
        case Left(y) => 
            a._1(y)
        case Right(z) => 
            a._2(z)
    }

(see this [post](https://alvinalexander.com/scala/how-to-define-use-partial-functions-in-scala-syntax-examples/) for more information on Scala partial functions).

But we must also show, or at least _test_, that both functions are mutual inverses, i.e. that:

`from(to(f)) == f`, for all `f: (Y => X, Z => X)`

`to(from(f)) == f`, for all `f: Either[Y, Z] => X`

We will perform some unit testing here with the following two functions:

In [ ]:
def ex1(f: Either[Boolean, Boolean]): Boolean = 
    false

val ex2: (Boolean => Boolean, Boolean => Boolean) = 
    (_ => false, _ => true)

thus fixing types $X$, $Y$ and $Z$ to $Boolean$. Then, we need the following equality functions:

In [ ]:
def equal0(f1: Boolean => Boolean, f2: Boolean => Boolean): Boolean = 
    f1(false) == f2(true) && 
    f1(true) == f2(true)

def equal1(f1: (Boolean => Boolean, Boolean => Boolean), 
           f2: (Boolean => Boolean, Boolean => Boolean)): Boolean = 
    equal0(f1._1, f2._1) && equal0(f2._2, f2._2)
    
def equal2(f1: Either[Boolean, Boolean] => Boolean, 
           f2: Either[Boolean, Boolean] => Boolean): Boolean = 
    f1(Left(false)) == f2(Left(false)) &&
    f1(Left(true)) == f2(Left(true)) &&
    f1(Right(false)) == f2(Right(false)) &&
    f1(Right(true)) == f2(Right(true))

and, finally, we can perform our test: 

In [ ]:
equal1(from(to(ex2)), ex2)
equal2(to(from(ex1)), ex1)